In [1]:
# Common modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
import scipy.sparse as sp
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# SVM specific modules
from sklearn import svm

In [2]:
BASE_PATH = os.path.join(os.getcwd(), os.pardir)
DATA_PATH = os.path.join(BASE_PATH, 'data')

In [3]:
train_df = pd.read_json(os.path.join(BASE_PATH, '01-milestone1', 'imputed_train.json'))
test_df = pd.read_json(os.path.join(DATA_PATH, 'test.json.zip'))

In [4]:
train_feats = sp.load_npz(os.path.join(DATA_PATH, 'training_feats.npz'))

test_feats = sp.load_npz(os.path.join(DATA_PATH, 'test_feats.npz'))

In [5]:
# X = train_df.drop(columns=['interest_level'])
y = train_df['interest_level']

X_sparse = train_feats

In [6]:
X_test = test_feats

In [7]:
X_sparse

<49308x35522 sparse matrix of type '<class 'numpy.float64'>'
	with 2626763 stored elements in Compressed Sparse Row format>

In [8]:
X_org = train_df.drop(columns=['interest_level', 'building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos','street_address'])

In [9]:
X_org

,bathrooms,bedrooms,latitude,listing_id,longitude,price
10,1.5,3,40.7145,7211212,-73.9425,3000
10000,1.0,2,40.7947,7150865,-73.9667,5465
100004,1.0,1,40.7388,6887163,-74.0018,2850
100007,1.0,1,40.7539,6888711,-73.9677,3275
100013,1.0,4,40.8241,6934781,-73.9493,3350
...,...,...,...,...,...,...
9999,1.0,2,40.7426,7098690,-73.9790,3200
99991,1.0,1,40.7102,6822449,-74.0163,3950
99992,1.0,1,40.7601,6881461,-73.9900,2595
99993,1.0,0,40.7066,6841891,-74.0101,3350


In [34]:
test_df = test_df.drop(columns=['building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos','street_address'])

In [35]:
test_df

,bathrooms,bedrooms,latitude,listing_id,longitude,price
0,1.0,1,40.7185,7142618,-73.9865,2950
1,1.0,2,40.7278,7210040,-74.0000,2850
2,1.0,0,40.7260,7174566,-74.0026,2295
3,1.0,2,40.7321,7191391,-74.0028,2900
5,1.0,1,40.7054,7171695,-74.0095,3254
...,...,...,...,...,...,...
124003,1.0,1,40.7925,6928108,-73.9454,1700
124005,1.0,2,40.7456,6906674,-73.9797,4195
124006,1.0,0,40.7416,6897967,-73.9829,2400
124007,2.0,2,40.7485,6842183,-73.9800,6895


## Preprocessing

In [10]:
# Convert labels from {low, medium high} -> {0, 1, 2}
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [14]:
#feature selection
# kbest = SelectKBest(mutual_info_classif, k=25).fit(X_sparse,y)
# X_fseltd = kbest.transform(X) 

NameError: name 'X' is not defined

In [13]:
# X_fseltd

NameError: name 'X_fseltd' is not defined

In [11]:
# X_test = kbest.transform(X_test)

In [12]:
# X_test

<74659x25 sparse matrix of type '<class 'numpy.float64'>'
	with 1000434 stored elements in Compressed Sparse Row format>

In [12]:
#Scale feature values
# X_org = preprocessing.scale(X_org)
# X_test = preprocessing.scale(X_test)

scaler = preprocessing.StandardScaler(with_mean=False)
X_normalized = scaler.fit_transform(X_org)

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## Train-test split

We will do 5-fold cross-validation

In [13]:
seed = 36201431

In [14]:
model = svm.SVC(kernel='rbf', random_state=seed, max_iter=100 ,probability=True)

In [15]:
svm_rbf = {'C': [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e2, 1e3, 1e4],
               'gamma': ['auto', 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e2, 1e3, 1e4]}
clf = GridSearchCV(model, svm_rbf)

In [16]:
scores = cross_validate(clf, X_normalized, y, cv=5, scoring='neg_log_loss')
# test_scores = -1 * scores['test_score']
# print(test_scores.mean().round(4))

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxS

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

...

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [22]:
scores
test_scores = -1 * scores['test_score']
train_scores = -1 * scores['train_score']
score = test_scores.mean().round(4)
train_score = train_scores.mean().round(4)
print("Mean log loss test  score: {0}".format(score))
print("Mean log loss train score: {0}".format(train_score))

Mean log loss test  score: 0.7888
Mean log loss train score: 0.787


In [25]:
model.fit(X_normalized, y)
# model.fit(X_train, y_train)

/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/ronniehung/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=100, probability=True, random_state=36201431,
  shrinking=True, tol=0.001, verbose=False)

In [36]:
y_pred = model.predict_proba(test_df)

In [38]:
def create_submission_csv(y_pred, X_test_indexes):
    df = pd.DataFrame(y_pred, columns=le.classes_)
    df.index = X_test_indexes
    df.index.name = 'listing_id'
    return df

In [40]:
X_test_indexes = test_df['listing_id']
output = create_submission_csv(y_pred, X_test_indexes)
output.to_csv('svm_predictions_2.csv')